In [8]:
import datetime, time
import json
import urllib.parse
import pandas as pd
import configparser
from pprint import pprint

import sqlite3

In [3]:
db_path = 'data/main.db'
db_table = 'testing'
db_col_names = ['doi', 'timestamp', 'doi_resolve_status',
                'doi_resolve_error', 'doi_url', 'fb_og_object',
                'fb_engagement', 'fb_response_error', 'am_response','am_response_error']
db_col_types = ['TEXT', 'TEXT', 'INTEGER',
                'TEXT', 'TEXT', 'TEXT',
                'TEXT', 'TEXT', 'TEXT', 'TEXT']
unique_col = "doi"

db_con = DBConnection(db_path=db_path,
                      db_table=db_table,
                      db_col_names=db_col_names,
                      db_col_types=db_col_types,
                      unique_col=unique_col)

In [92]:
con = sqlite3.connect("data/main.db")
df = pd.read_sql_query("SELECT * from testing", con)
df = df.set_index('doi')

In [93]:
def parse_json(row):
    try:
        row['am_response'] = json.loads(row['am_response'])
    except:
        pass

    try:
        row['fb_og_object'] = json.loads(row['fb_og_object'])
    except:
        pass
    
    try:
        row['fb_engagement'] = json.loads(row['fb_engagement'])
    except:
        pass
    return row
df = df.apply(parse_json, axis=1)

In [94]:
def extract_values(row):
    if row['fb_og_object']:
        row['fb_og_id'] = row['fb_og_object']['id']
    else:
        row['fb_og_id'] = None
    
    if row['fb_engagement']:
        row['fb_share_count'] = row['fb_engagement']['share_count']
        row['fb_reaction_count'] = row['fb_engagement']['reaction_count']
        row['fb_comment_plugin_count'] = row['fb_engagement']['comment_plugin_count']
        row['fb_comment_count'] = row['fb_engagement']['comment_count']
    else:
        row['fb_share_count'] = None
        row['fb_reaction_count'] = None
        row['fb_comment_plugin_count'] = None
        row['fb_comment_count'] = None
        
    try:
        row['am_id'] = row['am_response']['altmetric_id']
    except:
        row['am_id'] = None

    try:
        row['am_score'] = row['am_response']['score']
    except:
        row['am_score'] = None

    try:
        row['am_share_count'] = row['am_response']['counts']['facebook']['posts_count']
    except:
        row['am_share_count'] = None
    
    return row
df = df.apply(extract_values, axis=1)

In [99]:
df.describe()

,fb_share_count,fb_reaction_count,fb_comment_plugin_count,fb_comment_count,am_id,am_score,am_share_count
count,11.000000,11.000000,11.0,11.000000,2.600000e+01,36.000000,2.000000
mean,0.272727,0.181818,0.0,0.090909,4.227215e+06,3.268500,1.500000
std,0.467099,0.603023,0.0,0.301511,3.949929e+06,5.961745,0.707107
min,0.000000,0.000000,0.0,0.000000,1.875280e+05,0.000000,1.000000
25%,0.000000,0.000000,0.0,0.000000,1.768465e+06,0.000000,1.250000
50%,0.000000,0.000000,0.0,0.000000,2.899206e+06,1.000000,1.500000
75%,0.500000,0.000000,0.0,0.000000,5.348286e+06,3.125000,1.750000
max,1.000000,2.000000,0.0,1.000000,1.722374e+07,26.250000,2.000000
